# Earthquake Data Analysis

### Description

The catalog includes the magnitude, time of occurrence (s), and 3D coordinates (m) of earthquakes in about 20 years of recording in South California. Coordinates were converted from latitude, longitude, and depth of events in a seismic catalog. Magnitudes should be within the range $[0,8]$.

* **Waiting time (t)**: time interval between an event and the next one in the sequence.
* **Distance (r)**: Eucledian 3D distance between events. (each 3D set of coordinates refers to the hypocenter, i.e. the point triggering the slip in a fault that forms the earthquake)


### Assignments

1. Deduce what is the variable in each column of the catalog.
2. Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.
3. Compute the distribution $P_m(t)$ of waiting times for events of magnitude m or above (i.e. do not consider events below $m$). In shaping the bin sizes, take into account that this distribution is expected to have a power-law decay with time (e.g $\sim 1/t$), and that a power-law is well visualized in log-log scale. Do this analysis for many values of $m$, say $m=2,3,4,5$.
4. Compute the distribution $P_m(r)$ of the distance between an event and the next one, considering earthquakes of magnitude m or above. Also here make a clever choice for the bin sizes and try several values of $m$.
5. Compute the distribution $P_{m,R}(t)$ of waiting times for events of magnitude $m$ or above, which are separated by at most a distance $r<R$, for different values of m and $R$. (In this statistics, if the following event is farther than $R$, skip the $t$ and go to the next pair)
6. Eventually note if, from the analysis of the previous points, there emerges a scaling picture. Is there a suitable rescaling that collapses distributions for various $m$ (and eventually $R$ if point 5 is considered) on a single curve?

### Datasets

* column 1: index of the event
* column 2: index of the previous event that triggered it (defined with a given algorithm), -1 if no ancestor is found
* column 3: time (seconds) from 0:00 of Jan.1st, 1982
* column 4: magnitude
* columns 5, 6, and 7: 3D coordinates (meters) of the earthquake hypocenter, i.e. of the point from where it started. These Euclidean coordinates are derived from latitude, longitude and depth.

Joining each event to that with the index of the second column (if not -1), there emerges a set of causal trees.


### Contact
* Marco Baiesi <marco.baiesi@unipd.it>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

#PROJECTIONS LIBRARY
import pyproj

# Map library
import folium
from folium import plugins

#Colormap library
from branca.colormap import linear


In [2]:
header = ['ID', 'ID_previous_event', 'time', 'Magnitude', 'X', 'Y', 'Z']
df = pd.read_csv('./SouthCalifornia-1982-2011_Physics-of-Data.dat', sep=' ', names=header, index_col=0, nrows=100)
df.head()

,ID_previous_event,time,Magnitude,X,Y,Z
ID,,,,,,
0,-1,0.00000,2.71,-2571956,-4627162,3520602
1,0,36501.39072,2.12,-2363740,-4787011,3461373
2,0,37488.27744,2.33,-2363746,-4786942,3461232
3,0,47982.51648,2.57,-2475085,-4664024,3548479
4,0,60268.57056,2.98,-2238642,-4839098,3469546


In [3]:
time1 = datetime.datetime(1982, 1, 1, 0, 0, 0)
time2 = datetime.datetime(1970, 1, 1, 0, 0, 0)

delta = time1 - time2

total_seconds = delta.total_seconds()

#df["timestamp"] = pd.to_datetime(df["time"], unit="s").dt.strftime("%Y-%m-%d %H:%M:%S")
df["timestamp"] = pd.to_datetime(df["time"] + total_seconds, unit="s").dt.strftime("%Y-%m-%dT%H:%M:%S")

In [4]:
df.head()

,ID_previous_event,time,Magnitude,X,Y,Z,timestamp
ID,,,,,,,
0,-1,0.00000,2.71,-2571956,-4627162,3520602,1982-01-01T00:00:00
1,0,36501.39072,2.12,-2363740,-4787011,3461373,1982-01-01T10:08:21
2,0,37488.27744,2.33,-2363746,-4786942,3461232,1982-01-01T10:24:48
3,0,47982.51648,2.57,-2475085,-4664024,3548479,1982-01-01T13:19:42
4,0,60268.57056,2.98,-2238642,-4839098,3469546,1982-01-01T16:44:28


In [5]:
# Definition of the two reference frames 
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84') #euclidean
lla  = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84') #latlon

# Coordinates conversion
points = np.array([df.X, df.Y, df.Z])
gcoor  = pd.DataFrame(zip(*(pyproj.transform(ecef, lla, points[0], points[1], points[2], radians=False))))

gcoor.columns = ['longitude', 'latitude', 'depth']
df = pd.concat([df, gcoor], axis=1)

/tmp/ipykernel_9519/1083525435.py:7: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  gcoor  = pd.DataFrame(zip(*(pyproj.transform(ecef, lla, points[0], points[1], points[2], radians=False))))


$$X = (N(\phi)+h) \space cos\phi \space cos\lambda $$
$$Y = (N(\phi)+h) \space cos\phi \space sin\lambda $$
$$Z = \left( \frac {b^2} {a^2} N(\phi) + h \right) sin\phi $$

where:

$$ N(\phi) = \frac {a^2} {\sqrt{a^2 cos\phi^2 + b^2 sin\phi^2}} = 
\frac {a} {\sqrt{1 - e^2 sin\phi ^2}}
$$




Source: <a href="https://en.wikipedia.org/wiki/Geographic_coordinate_conversion#:~:text=In%20geodesy%2C%20geographic%20coordinate%20conversion,translation%20among%20different%20geodetic%20datums.">Geographic Coordinate Conversion</a>

In [6]:
df.head()

,ID_previous_event,time,Magnitude,X,Y,Z,timestamp,longitude,latitude,depth
0,-1,0.00000,2.71,-2571956,-4627162,3520602,1982-01-01T00:00:00,-119.067002,33.803070,-13866.339243
1,0,36501.39072,2.12,-2363740,-4787011,3461373,1982-01-01T10:08:21,-116.279332,33.133255,-9094.154541
2,0,37488.27744,2.33,-2363746,-4786942,3461232,1982-01-01T10:24:48,-116.279718,33.132481,-9220.807091
3,0,47982.51648,2.57,-2475085,-4664024,3548479,1982-01-01T13:19:42,-117.953814,34.081790,-9788.561413
4,0,60268.57056,2.98,-2238642,-4839098,3469546,1982-01-01T16:44:28,-114.825995,33.229464,-10453.018622


In [9]:
# DON'T EXECUTE THIS CELL WITH A LOT OF DATA MAX: 3000 ROWS.

m = folium.Map(location=[35.296933, -119.417933], zoom_start=6)

folium.TileLayer("Stamen Terrain").add_to(m)

from branca.colormap import linear

linear.YlOrRd_09.colors.reverse()

colormap = linear.YlOrRd_09.scale(-32000, 0)
colormap = colormap.to_step(index=[-32000, -25000, -20000, -15000, -10000, -5000, 0])
colormap.caption = "Depth of the hypocenter (m)"
colormap.add_to(m)

# THIS WORKS BUT ONLY SHOWS THE POINTS WITHOUT TAKING IN CONSIDERATION THE TIME. COMMENT THE OTHER CODE
# FROM EARTHQUAKES = [] TO ).ADD_TO(M) TO SEE IT 

#for i,row in df.iterrows():
#    color = colormap(row["depth"] )
#    folium.CircleMarker(location = [row["latitude"], row["longitude"]], radius=row["Magnitude"]*3, 
#                        color=color, fill=True, fill_color=color,
#                        fill_opacity=1, popup= row["depth"]).add_to(m)


#THIS DOESN'T WORK FOR NOW IT SHOWS THE TIME LAPSE BUT THE DATA DOESN'T APPEAR :( 

earthquakes = []
for i, row in df.iterrows():
    color = colormap(row["depth"])
    earthquakes.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["latitude"], row["longitude"]]
        },
        "properties": {
            "time": row["timestamp"],
            "style": {"radius": row["Magnitude"]*10, "color" : color, "fill": True, 
                      "fill_color": color, "fill_opacity": 1                   
            }
        }
    })

folium.plugins.TimestampedGeoJson(
    {
    "type": "FeatureCollection",
    "features": earthquakes
    },
    period="P1D", duration = "P1Y",
    add_last_point=True, 
).add_to(m)
        
        


m